In [1]:
import json
from tqdm import tqdm

In [2]:
with open('weak-negatives.json') as fopen:
    weak = json.load(fopen)
    
len(weak)

1210878

In [3]:
len(weak)

1210878

In [4]:
import cleaning

In [5]:
import re

def preprocessing(string):
    string = re.sub(
        'http\S+|www.\S+',
        '',
        ' '.join(
            [i for i in string.split() if i.find('@') < 0]
        ),
    )
    
    chars = ',.()!:\'"/;=-'
    for c in chars:
        string = string.replace(c, f' {c} ')
        
    string = re.sub(
        u'[0-9!@#$%^&*()_\-+{}|\~`\'";:?/.>,<]',
        ' ',
        string,
        flags = re.UNICODE,
    )
    string = re.sub(r'[ ]+', ' ', string).strip()
    
    return string.lower()

In [6]:
cleaned = []

for i in tqdm(range(len(weak))):
    c = preprocessing(weak[i])
    cleaned.append((weak[i], c))

100%|██████████| 1210878/1210878 [00:26<00:00, 44959.73it/s]


In [7]:
negate_words = {
    'tak',
    'jangan',
    'tidak',
    'enggak',
    'tiada',
    'bukan',
    'usah',
    'tidaklah',
    'jgn',
    'tk',
    'bkn',
    "shouldnt",
    "dont",
    "doesnt",
}

import itertools

def _pad_sequence(
    sequence,
    n,
    pad_left = False,
    pad_right = False,
    left_pad_symbol = None,
    right_pad_symbol = None,
):
    sequence = iter(sequence)
    if pad_left:
        sequence = itertools.chain((left_pad_symbol,) * (n - 1), sequence)
    if pad_right:
        sequence = itertools.chain(sequence, (right_pad_symbol,) * (n - 1))
    return sequence

def ngrams(
    sequence,
    n: int,
    pad_left = False,
    pad_right = False,
    left_pad_symbol = None,
    right_pad_symbol = None,
):
    sequence = _pad_sequence(
        sequence, n, pad_left, pad_right, left_pad_symbol, right_pad_symbol
    )

    history = []
    while n > 1:
        try:
            next_item = next(sequence)
        except StopIteration:
            return
        history.append(next_item)
        n -= 1
    for item in sequence:
        history.append(item)
        yield tuple(history)
        del history[0]

In [8]:
with open('populate-results.json') as fopen:
    l = json.load(fopen)

In [9]:
n, p = [], []

for k, v in l.items():
    if v == 'negative':
        n.append(k)
    else:
        p.append(k)
        
len(n), len(p)

(2260, 2922)

In [10]:
negatives, shouldnot = n[:], []
for w in negate_words:
    for w_ in p:
        if w_ in ['sabar']:
            continue
        negatives.extend([f'{w}{w_}', f'{w} {w_}'])
    for w_ in n:
        shouldnot.extend([f'{w}{w_}', f'{w} {w_}'])

In [11]:
negatives_words = set(negatives)
shouldnot_words = set(shouldnot)

In [12]:
laughing = {
    'huhu',
    'haha',
    'gagaga',
    'hihi',
    'wkawka',
    'wkwk',
    'kiki',
    'keke',
    'huehue',
    'hshs',
    'zzz',
    'hoho',
    'hew',
    'uwu',
    'ooo',
    'sksk',
    'ksks',
    'gitu',
    'meow',
    'alhamdulillah',
    'muah',
    'mmuahh',
    'hehe',
    'salamramadhan',
    'happywomensday'
}

In [13]:
reject = ['sukanya', 'tolong', 'bulan', 'bau', 'pukul', 'handai', 'abis', 'visi',
'hiburkan', 'kek', 'depa', 'senyum', 'ulama', 'habis', 'bener', 'dipilih',
'suami', 'penyertaan', 'teknikal', 'jam', 'mencapai', 'kekuatan', 'hepi']

negatives_words = {w for w in negatives_words if w not in reject}

In [14]:
ns, found = [], []
for s in tqdm(cleaned[:10000]):
    splitted = s[1].split()
    ngs = splitted[:]
    ngs.extend([' '.join(n) for n in ngrams(splitted, 2)])
    r = set(ngs) & negatives_words
    sn = set(ngs) & shouldnot_words
    string = [
        word
        for word in splitted
        if any([laugh in word for laugh in laughing])
    ]
    if len(r) and not len(sn) and not len(string):
        found.extend(list(r))
        ns.append(s[0])
        print(r, s[1])

  9%|▉         | 898/10000 [00:00<00:02, 4428.47it/s]

{'ngaku'} hayoo ngaku sebenernya kamu suka juga kan sama aku
{'stiap'} sy pnya anak usia thn dia suka sekali berkuda stiap ke kebun binatang yg di tujunya pasti
{'tak suka'} antara kawan kita mesti ada sorang yang tak suka bawa beg lepastu suka tumpang barang dia dalam beg orang lain ak
{'sanggup'} punya pasal lah i takleh move onnnn sanggup repeat dekat iflix angkara percaya tergadai bahagia nur
{'nangis', 'lemah'} padahal lu tau kan klo gua sama nadinne cewek cengeng lemah tukang nangis bahkan lu gak suka denger kita nang
{'gurau', 'tak lawak', 'curang'} tak lawak k agak laki suka ke perempuan curang takkan agak nak gurau ada byk lg nak gurau k baii
{'gatau'} katanya cewek juga perlu gengsi dan jual mahal tapi gatau atuh da aku ma gini kalo udah suka yaudah gabisa di
{'berasa'} dari dulu suka ini lagunya soft asik aja terusnya berasa lagi di karpet terbang kuk
{'liberal'} aku makin suka malaysia yg liberal mcm ni pondan g umrah hukum faraid pakai undang sendiri adik ipar boleh peluk 

 17%|█▋        | 1694/10000 [00:00<00:01, 4181.20it/s]

orang yang paling rugi di dunia ini ialah orang yang suka bertangguh sebab masa itu emas
{'mati', 'rokok'} sblum corona msuk gak tkut mati ngebut suka nerobos lampu merah rokok membunuhmu tpi diisep trs gak peduli kesehatan stlah corona msuk tiba tkut mati karena corona tiba peduli kesehatan beli masker orang indonesia emang suka tiba tiba sayang tiba pergi ninggalin
{'fandom'} [comifuro ] trade comifuro for danmei only klo mau dua duany untuk trade postcard oc fandom lain cuma bs available yg hualian nya akuh ga punya oc capek gambar jg with snacks juga oke aku suka strawberry www
{'penderitaan', 'gila', 'curang'} i ikut you balik rumah lah gila bella ni sekali try je dah dapat kesian tunang dia wan guys setia lah ya dengan pasangan masing masing focus bina hidup together until tua no benefits kalau curang you takkan bahagia atas penderitaan orang lain
{'menyayangi'} aduh mamak seneng nih sama yg suka menyayangi
{'mikir'} hidup emang gak bisa ditebak ya entah yang hari ini akan hadir 

 25%|██▌       | 2511/10000 [00:00<00:01, 4138.50it/s]

...

 jelek nya aku suka ngejugde selera musik orang selera pakaian orang padahal mah tiap orang kan punya selera masing masing ya
{'jahat'} kdm aku suka kim jaewook pas jadi mo taegu ih kok suka sama dia disitu sih kan dia jadi orang jahat
{'share'} maybe ada yg dah tahu tapi nak share jugak sebab mungkin berguna pada yg lain benda ni aku beli kat daiso kalau sesiapa yg suka jalan sorang sorang better beli ni untuk keselamatan diri if anything happen cabut je kat mulut burung tu pastu nnt dia akan bunyi dgn kuat
{'tak reti', 'mati', 'malu'} suami mati family and adik beradik suami yang kelapaq harta sikit pun tak heran dekat balu we hear this kind of story over and over again tapi melayu jenis tak reti malu camna entah suka buat perangai ni
{'menangi'} anda suka dunia fotografi jom ambil gambar amp menangi smartphone huawei terbaharu
{'gila'} tak tau ah kenapa aku suka gila buat edit format macam word tu pastu kalau coding suka main dengan style design
{'terlepas'} suka ga suka terima ga t

 91%|█████████ | 9116/10000 [00:02<00:00, 3355.50it/s]

 i ikut you balik rumah lah gila bella ni sekali try je dah dapat kesian tunang dia wan guys setia lah ya dengan pasangan masing masing focus bina hidup together until tua no benefits kalau curang you takkan bahagia atas penderitaan orang lain
{'usah suka'} pernah suka sama cowok udah sering chat an tapi tiba dia bilang gak usah suka korea nonton anime saja bye paling serius
{'aneh', 'sedih', 'melulu'} suka kpop ga melulu soal dia yang selalu menikmati visual idol suka kpop justru ngebuat hidup sebagian orang lebih berwarna dan bermakna disaat lagi sedih dan sama sekali ga ada teman buat curhat justru lagu korea yg kalian anggap aneh itu menjadi teman kami dlm kesunyian c
{'palsu', 'busy'} ni setuju but sampai certain point saja sbb masyarakat kita ramai busy body amp suka sebar berita palsu yg rapat dg
{'silap'} aku la silap boleh tarik balik x nk bg messi messi suka pergi kerja
{'kayak', 'garong', 'peka'} aku suka kamu tapi kamu gak peka ini tuh intinya snap x a snap x i snap x u sna

100%|██████████| 10000/10000 [00:02<00:00, 3867.32it/s]

 am i too old bc ive tried alll paling tak suka nano nano paling suka omg semua kot balik ni nak beli biskut
{'sepele', 'merasa'} sebenernya kl udah suka sm orgnya pertanyaan sesepele apapun akan merasa gpp kl bosen kita akan effort jg cari topik yg kita mau coba obrolin kl udah dasarnya ga tertarik bgt ya jd gini pertanyaan sepele terlihat annoying dan kita ga mau effort cari topik lain jd nyalahin
{'dadah'} sape je suka org hisap dadah tapi aku x hisap dadah pun org x suke aku
{'mengenalkan', 'aneh', 'malu'} bu aku ingin mengenalkan perempuan kepadamu anaknya riang ceria lucu lagi tapi bu kalo di hadapanmu ia pasti akan diam amp lebih banyak menarik ujung bajuku bu ibu jangan nanya yang aneh aneh yah bu sebab ia kalo sedang malu wajah cantiknya menjadi memerah manja
{'share'} korang suka beli juadah apa untuk berbuka puasa masa dekat bazar jom share admin paling suka murtabak dan tepung
{'dosa'} sebagian dari kita suka membahas dosa riya teman kita yang posting barang mahal di sosial

In [21]:
reject = ['mencintai', 'seindah', 'mohon', 'dipuji', 'share', 'keberanian', 'disukai', 'tulus',
         'heh', 'rok', 'lengkap', 'bersuka', 'berharap', 'memantapkan', 'minta', 'menemui', 'share',
         'dipake', 'buih', 'basuh', 'die']

negatives_words = {w for w in negatives_words if w not in reject}

In [22]:
additional = {'terutama', 'suka suka', 'sunnah nabi', 'ikut sunnah', 'suka banget', 'hehe rt',
             'masih suka', 'sama suka', 'suka juga', 'suka sekali', 'dia sedap', 'berhak untuk',
             'akan membantu', 'suka jbjb', 'kena percaya', 'erti syukur', 'akan membela',
             'pun terharu'}

shouldnot_words = shouldnot_words | additional

In [23]:
ns, found = [], []
for s in tqdm(cleaned):
    splitted = s[1].split()
    ngs = splitted[:]
    ngs.extend([' '.join(n) for n in ngrams(splitted, 2)])
    r = set(ngs) & negatives_words
    sn = set(ngs) & shouldnot_words
    string = [
        word
        for word in splitted
        if any([laugh in word for laugh in laughing])
    ]
    if len(r) and not len(sn) and not len(string):
        found.extend(list(r))
        ns.append(s[0])

100%|██████████| 1210878/1210878 [01:02<00:00, 19513.88it/s]


In [24]:
len(ns) / len(cleaned)

0.9301482065080049

In [26]:
with open('weak-negatives.json', 'w') as fopen:
    json.dump(ns, fopen)